In [1]:
from pathlib import Path

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import os
from tqdm import tqdm
import numpy as np

import torchvision.transforms as v1
from torchvision.io import read_image

plt.rcParams["savefig.bbox"] = 'tight'
torch.manual_seed(1)

In [2]:
path = 'S:\drone_dataset_v4.0_pictures_spectr'
dirnames = os.listdir(path)
np.random.shuffle(dirnames)
print(len(dirnames))

33


In [3]:
!pip install mlconfig

In [4]:
import torch, torchvision
from torch.utils.data.dataloader import DataLoader
from sklearn.model_selection import train_test_split

import os, shutil
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import mlconfig
from importlib import import_module
from torch.utils.data import Dataset, DataLoader

def load_function(attr):
    module_, func = attr.rsplit('.', maxsplit=1)
    return getattr(import_module(module_), func)

config = mlconfig.load('config_resnet18.yaml')

In [5]:
class MyDataset(Dataset):
    def __init__(self, X, y):

      self.X = X
      self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        x  = torch.tensor(self.X[idx], dtype=torch.float32)
        y = torch.tensor(self.y[idx], dtype=torch.float32)
        return x, y

In [9]:
torch.cuda.empty_cache()
model = load_function(config.model.architecture)(pretrained=False)
model.fc = torch.nn.Linear(in_features=512, out_features=1, bias=True)
print(model)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

import numpy as np
import copy, timeit

optimizer = load_function(config.optimizer.name)(model.parameters(), lr=config.optimizer.lr)
criterion = load_function(config.loss_function.name)()
scheduler = load_function(config.scheduler.name)(optimizer, step_size=config.scheduler.step_size, gamma=config.scheduler.gamma)

if device!='cpu':
    model = model.to(device)

val_loss = []
val_acc = []
train_loss = []
train_acc = []
epochs = config.epoch

best_acc = 0
best_model = copy.deepcopy(model.state_dict())
ind = 0

start = timeit.default_timer()
for epoch in range(1, len(dirnames)+1):
    data = []
    labels = []
    for dir in dirnames[ind:ind+1]:
        if dir.endswith('csv'):
            p = 0
            continue
        p=1
        for file in tqdm(os.listdir(path+'/'+dir)):
            data_file = np.asarray(np.load(path + '/' + dir + '/' + file, 'r+'))
            data.append(data_file)
            if 'drone' in list(file.split('_')):
                labels.append(0.0)
            if 'noise' in list(file.split('_')):
                labels.append(0.5)
            if 'wifi' in list(file.split('_')):
                labels.append(1.0)
    ind+=1
    if p == 0:
        continue
    data = np.asarray(data)
    labels = np.asarray(labels)
    

    X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.3, random_state=12)
    train = MyDataset(X_train, y_train)
    val = MyDataset(X_val, y_val)
    batch_size = 3
    train_dl = DataLoader(train, batch_size=batch_size, shuffle=True)
    val_dl = DataLoader(val, batch_size=batch_size, shuffle=True)
    
    print(f"Epoch : {epoch}\n")
    dataloader = None

    for phase in ['train', 'val']:
        loss = 0
        correct = 0
        total = 0
        batch_num = 0
        
        if phase == 'train' :
            model.train()
            dataloader = train_dl
        else:
            model.eval()
            dataloader = val_dl

        for (img, label) in enumerate(dataloader):
            img = torch.unsqueeze(img,1)
            #label = torch.unsqueeze(label,1)
            img, label = img.to(device), label.to(device)
            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                output = model(img)
                _, pred = torch.max(output.data, 1)
                loss = criterion(output, label)
                #with torch.autocast('cuda'):
                 #   loss = criterion(torch.tensor(output).cuda(), torch.tensor(label).cuda())
                if phase=='train' :
                    loss.backward()
                    optimizer.step()
            
            loss += loss.item() * img.size(0)
            torch.cuda.empty_cache()
            correct += torch.sum(pred == label.data)
            total += label.size(0)
            batch_num += 1
            if batch_num % 50 == 0:
                print(f"Epoch : {epoch}\t{phase} batch {batch_num} completed")
                
        if phase=='train' :
            train_loss.append(loss/len(train_dl))
            train_acc.append(correct/total)
            scheduler.step()
            print(f'{phase} Loss: {train_loss[-1]:.4f}\tAccuracy: {train_acc[-1]:.4f}')
        else :
            val_loss.append(loss/len(val_dl))
            val_acc.append(correct/total)
            print(f'{phase} Loss: {val_loss[-1]:.4f}\tAccuracy: {val_acc[-1]:.4f}')

            if val_acc[-1] > best_acc :
                best_acc = val_acc[-1]
                best_model = copy.deepcopy(model.state_dict())

        torch.cuda.empty_cache()
    
    print()

end = timeit.default_timer()
print(f"Total time elapsed = {end - start} seconds")
name = 'model2.pth'
model.load_state_dict(best_model)
model = model.to('cpu')
torch.save(model.state_dict(), name)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

100%|██████████████████████████████████████████████████████████████████████████████| 248/248 [00:00<00:00, 1720.65it/s]


MemoryError: Unable to allocate 2.03 GiB for an array with shape (173, 3, 1024, 1024) and data type float32

In [ ]:
train_loss = [i.item() for i in train_loss]
val_loss = [i.item() for i in val_loss]
train_acc = [i.item() for i in train_acc]
val_acc = [i.item() for i in val_acc]

import matplotlib.pyplot as plt
plt.figure()
plt.plot(range(1,epochs+1), train_loss, color='blue')
plt.plot(range(1,epochs+1), val_loss, color='red')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss Curve')
plt.legend(['Train Loss', 'Validation Loss'])
plt.show()

import matplotlib.pyplot as plt
plt.figure()
plt.plot(range(1,epochs+1), train_acc, color='blue')
plt.plot(range(1,epochs+1), val_acc, color='red')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy Curve')
plt.legend(['Train Accuracy', 'Validation Accuracy'])
plt.show()